# Creating and Valuing a CDS Contract

Replicating Markit Pricing on 20 Aug 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.products.credit import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Creating a CDS Contract

In [3]:
tradeDate = FinDate(20, 8, 2020)
effectiveDate = FinDate(21, 8, 2020)
maturityDate = FinDate(20, 6, 2025)
runningCoupon = 0.05
notional = ONE_MILLION
longProtection = True

In [4]:
cdsContract = FinCDS(effectiveDate, maturityDate, runningCoupon, notional, longProtection)

## Build Libor Curve

In [5]:
settlementDate = FinDate(24, 8, 2020)

In [14]:
dcType = FinDayCountTypes.ACT_360
depo1 = FinLiborDeposit(settlementDate, "1M", 0.001709, dcType)
depo2 = FinLiborDeposit(settlementDate, "2M", 0.002123, dcType)
depo3 = FinLiborDeposit(settlementDate, "3M", 0.002469, dcType)
depo4 = FinLiborDeposit(settlementDate, "6M", 0.003045, dcType)
depo5 = FinLiborDeposit(settlementDate, "12M", 0.004449, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = FinLiborSwapTypes.PAYER
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"2Y", swapType, 0.002155,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"3Y", swapType, 0.002305,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"4Y", swapType, 0.002665,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"5Y", swapType, 0.003290,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"6Y", swapType, 0.004025,fixedFreq,dcType)
swap6 = FinLiborSwap(settlementDate,"7Y", swapType, 0.004725,fixedFreq,dcType)
swap7 = FinLiborSwap(settlementDate,"8Y", swapType, 0.005430,fixedFreq,dcType)
swap8 = FinLiborSwap(settlementDate,"9Y", swapType, 0.006075,fixedFreq,dcType)
swap9 = FinLiborSwap(settlementDate,"10Y", swapType, 0.006640,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

# Build a CDS Curve

In [15]:
cdsSpread = 0.01

In [16]:
cds1 = FinCDS(settlementDate, "6M", cdsSpread)
cds2 = FinCDS(settlementDate, "1Y", cdsSpread)
cds3 = FinCDS(settlementDate, "2Y", cdsSpread)
cds4 = FinCDS(settlementDate, "3Y", cdsSpread)
cds5 = FinCDS(settlementDate, "4Y", cdsSpread)
cds6 = FinCDS(settlementDate, "5Y", cdsSpread)
cds7 = FinCDS(settlementDate, "7Y", cdsSpread)
cds8 = FinCDS(settlementDate, "10Y", cdsSpread)

In [17]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [18]:
recoveryRate = 0.40

In [19]:
issuerCurve = FinCDSCurve(settlementDate, cdss, liborCurve, recoveryRate)

In [20]:
print(issuerCurve)

OBJECT TYPE: FinCDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.5698630,  0.9903756
 1.0739726,  0.9819794
 2.0739726,  0.9655268
 3.0739726,  0.9493544
 4.0767123,  0.9334122
 5.0767123,  0.9177837
 7.0767123,  0.8873181
10.0794521,  0.8434794


In [21]:
cdsContract.printFlows(issuerCurve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
MON 21 SEP 2020   0.252778     12638.89     0.999867     0.998699     12620.77
MON 21 DEC 2020   0.252778     12638.89     0.999098     0.994482     12557.82
MON 22 MAR 2021   0.252778     12638.89     0.998024     0.990284     12491.35
MON 21 JUN 2021   0.252778     12638.89     0.996547     0.986123     12420.46
MON 20 SEP 2021   0.252778     12638.89     0.995695     0.981979     12357.70
MON 20 DEC 2021   0.252778     12638.89     0.996322     0.977851     12313.51
MON 21 MAR 2022   0.252778     12638.89     0.996629     0.973741     12265.52
MON 20 JUN 2022   0.252778     12638.89     0.996090     0.969648     12207.35
TUE 20 SEP 2022   0.255556     12777.78     0.995520     0.965527     12282.01
TUE 20 DEC 2022   0.252778     12638.89     0.994896     0.961469     12089.88
MON 20 MAR 2023   0.250000     12500.00     0.994268     0.957473     11899.81
TUE 20 JUN 2023   0.255556     12777.78     0.993592   

# Valuation

In [22]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD   99.99955 bp


In [23]:
cdsContract.valueFastApprox(settlementDate, 0.004, 0.01, 0.40, 0.40)

(-195533.6454106767, -187200.31207734335, 533.6950180429558, 44.32197353540687)

In [24]:
v = cdsContract.value(settlementDate, issuerCurve, recoveryRate)

In [25]:
full_pv = v['full_pv'] 
clean_pv = v['clean_pv']

In [26]:
print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -195421.00
CLEAN VALUE   -187087.66


In [27]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.708758


In [28]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 60


In [29]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON -8333.333333333334


In [30]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 46771.65011225943


In [31]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 242192.6470217891


In [32]:
cdsContract.riskyPV01(settlementDate, issuerCurve)

{'full_rpv01': 4.843852940435782, 'clean_rpv01': 4.677186273769115}

## Risk Measures

In [33]:
cdsContract.creditDV01(settlementDate, issuerCurve, recoveryRate)

542.5032361687918

In [34]:
cdsContract.interestDV01(settlementDate, issuerCurve, recoveryRate)

46.52493245183723

Copyright (c) 2020 Dominic O'Kane